In [10]:
import pandas as pd
import seaborn as sns
import json

In [19]:
def get_result_df(result_dir):
    with open(f'{result_dir}/results.json', 'r') as f:
        result = json.load(f)
        
    refined_result = {k: result[k]['cv_avg_scores'] for k in result.keys()}
    return pd.DataFrame.from_dict(refined_result).T

In [11]:
baseline_no_imputation_mutual = '../baseline_no_imputation_mutual_info_classif'
target_metric = 'AUC'

get_result_df(baseline_no_imputation_mutual)

In [13]:
result_dir = '../chained_mutual_info_classif'
target_metric = 'AUC'
get_result_df(baseline_no_imputation_mutual)

In [20]:
result_dir = '../chained_chi2'
target_metric = 'AUC'
get_result_df(baseline_no_imputation_mutual)

,fit_time,score_time,test_precision,test_recall,test_f1,test_AUC
rf,27.820193,1.364888,0.938408,0.952158,0.945215,0.988822
lr,1.206974,0.008197,0.823825,0.877513,0.849804,0.925940
svm,71.635541,4.395177,0.963351,0.986658,0.974859,0.988258
ada,110.741588,3.290752,0.824877,0.862642,0.843321,0.924301
et,24.077533,0.831436,0.953154,0.967910,0.960454,0.992853
xgb,54.526390,0.243923,0.970823,0.969320,0.970062,0.995736
knn,0.010593,3.914428,0.896379,0.985483,0.938793,0.987155
